# Setup


In [1]:
import sys
from google.colab import drive, files

In [2]:
%%bash 
pip install git+https://github.com/lucasb-eyer/pydensecrf.git
pip install --upgrade setuptools
pip install kaggle
pip install gdown
pip install pytorch-lightning

  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-vv3arikm
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pydensecrf: filename=pydensecrf-1.0rc2-cp37-cp37m-linux_x86_64.whl size=2774221 sha256=2ce66790c2f299ce0a341c4c9043fd7d52fb3e445877a79190d332d979825a52
  Stored in directory: /tmp/pip-ephem-wheel-cache-jny2ggq4/wheels/81/ac/ed/09f1890b954ca1ff7f189518cc9a319138e50e7ea88937fae9
Successfully built pydensecrf
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (56.0.0)
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=f8e0a71da73752f5cde6f5279ee7bc5ad45ae14ccf7b5ca7c8bbcac29b6b5c51
  Stored in directory: /ro

  Running command git clone -q https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-vv3arikm


In [3]:
# upload kaggle API key
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"giorgio123213","key":"71d614caaf97835377e7e46121d1dd90"}'}

In [4]:
# download DSTL data
%%bash
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
kaggle competitions download -c 'dstl-satellite-imagery-feature-detection' -p ../../ -f sixteen_band.zip
kaggle competitions download -c 'dstl-satellite-imagery-feature-detection' -p ../../ -f train_wkt_v4.csv.zip
kaggle competitions download -c 'dstl-satellite-imagery-feature-detection' -p ../../ -f grid_sizes.csv.zip
unzip -q ../../grid_sizes.csv.zip -d /content/
unzip -q ../../train_wkt_v4.csv.zip -d /content/
unzip -q ../../sixteen_band.zip -d /content/

100%|##########| 7.30G/7.30G [01:22<00:00, 95.3MB/s]
100%|##########| 11.1M/11.1M [00:00<00:00, 65.2MB/s]
100%|##########| 2.17k/2.17k [00:00<00:00, 1.81MB/s]


In [5]:
# hTorch library
%%bash
git clone https://github.com/giorgiozannini/hTorch.git # cloning my fork before merging with master
cd hTorch
python setup.py install
cd ..

running install
running bdist_egg
running egg_info
creating htorch_giorgiozannini.egg-info
writing htorch_giorgiozannini.egg-info/PKG-INFO
writing dependency_links to htorch_giorgiozannini.egg-info/dependency_links.txt
writing top-level names to htorch_giorgiozannini.egg-info/top_level.txt
writing manifest file 'htorch_giorgiozannini.egg-info/SOURCES.txt'
writing manifest file 'htorch_giorgiozannini.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/htorch
copying htorch/layers.py -> build/lib/htorch
copying htorch/quaternion.py -> build/lib/htorch
copying htorch/utils.py -> build/lib/htorch
copying htorch/functions.py -> build/lib/htorch
copying htorch/__init__.py -> build/lib/htorch
creating build/lib/tests
copying tests/__init__.py -> build/lib/tests
copying tests/quaternion_test.py -> build/lib/tests
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64

Cloning into 'hTorch'...
zip_safe flag not set; analyzing archive contents...


In [4]:
sys.path.append('hTorch/')

# Hyperparameters



In [7]:
# hyperparameters settings
!cat hTorch/qpsp/constants.py

# Dataset options
BATCH_SIZE = 2
SHUFFLE = False
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.5

# Training options
DEEP_SUPERVISION = False
DATA_SIZE_TRAIN = 1000
DATA_SIZE_VAL = 100
DROPOUT = 0.1
CLASSES = 10
LAYERS = 101
LR = 3e-4
NUM_EPOCHS = 20
ALPHA_AUX = 0

# Loss parameters
ALPHA = 0.7
BETA = 0.3
GAMMA = 0.75

# CRF parameters
MAX_ITER = 2
POS_W = 3
POS_XY_STD = 3
BI_XY_STD = 3
BI_RGB_STD = 3
BI_W = 3

# Run

In [208]:
import pytorch_lightning as pl
from qpsp.qpsp import PSPNet
from qpsp.dstl_dataset import LitDSTL
import torch

In [209]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = PSPNet().to(device)
dm = LitDSTL()
trainer = pl.Trainer(gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [210]:
trainer.fit(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layer0 | Sequential | 115 K 
1 | layer1 | Sequential | 236 K 
2 | layer2 | Sequential | 1.2 M 
3 | layer3 | Sequential | 7.1 M 
4 | layer4 | Sequential | 15.0 M
5 | ppm    | PPM        | 4.2 M 
6 | cls    | Sequential | 52.4 M
7 | aux    | Sequential | 2.4 M 
--------------------------------------
82.6 M    Trainable params
0         Non-trainable params
82.6 M    Total params
330.522   Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


KeyboardInterrupt: ignored